In [1]:
import sys
import pandas as pd
sys.path.append('..')

# Подготовка данных

Загрузка данных

In [2]:
xlsx_file = pd.ExcelFile('../input/БОГРАД_PISY.xlsx')
trees = ['pisy_01a', 'pisy_01b', 'pisy_02a', 'pisy_03a', 'pisy_07a', 'pisy_12b', 'pisy_14a']
columns = {_:f'D{__}' if __<16 else f'CWT{__-15}' for _, __ in zip(range(2,32), range(1,31))}
columns[0] = 'Tree'
columns[1] = 'Year'

Функция нормализации трахеид

In [ ]:
def get_normalized_list(x: list, norm : int):
    """
    Функция получения нормированного списка
    :param x: список для нормирования
    :param norm: норма
    :return: l_norm - нормированный к e список l
    """
    l_raw = []  # промежуточный список
    n = len(x)
    for i in range(n):
        for j in range(norm):
            l_raw += [x[i]]
    l_norm = []
    for i in range(norm):
        l_norm += [1 / n * sum([l_raw[j] for j in range(n * i, n * (i + 1))])]
    return l_norm

Нормализуем входные данные

In [ ]:
dataframes = []
for tree in trees:
    df = xlsx_file.parse(tree)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(axis=0)

    norm_traches = dict()
    for year in set(df['Год']):
        norm_traches[int(year)] = [tree, int(year)] + get_normalized_list(list(df[df['Год']==year]['Dmean']), 15)+ get_normalized_list(list(df[df['Год']==year]['CWTmean']), 15)
    
    dataframes += [pd.DataFrame(norm_traches).transpose().rename(columns=columns).reset_index(drop=True)]

df = pd.concat(dataframes).reset_index(drop=True)

Сохраняем их в .xlsx файл

In [ ]:
df.to_excel('../output/Bograd_PISY_normalized.xlsx', index=False)

Входные множества:

$T=\left\{ t_1, ..., t_2 \right\}$ — Множество деревьев

$Y(t)=\left\{ y_{t1}, ..., y_{tn_t} \right\}$ — Множество лет, за которые есть измерения, для дерева $t$,  $t \in T$

$Y=\bigcup_{t \in T} Y(t)$ — Множество всех лет, за которые есть измерения

$T(y)=\left\{ t_{y1}, ..., t_{ym_y} \right\}$ — Множество деревьев, по которым есть измерения в год $y$, $y \in Y$

Объекты:

$R(t,y) = \left\{ d_1, ... , d_{15}, c_1, ..., c_{15}\right\}$ — Нормированная к 15 трахеида.

$d_i=d_i(t,y)$ — Диаметр $i$-ой клетки

$c_i=c_i(t,y)$ — Толщина клеточной стенки $i$-ой клетки

$i=\overline{1,15}$, $t\in T, y\in Y(t)$

Рассчёт средних значений трахеид по годам и по деревьям:

In [ ]:
mean_objects_years = dict()

for year in set(df['Year']):
    temp_data = df[df['Year']==year]
    if len(temp_data) > 3:
        mean_objects_years[year] = temp_data.mean()[1:]


mean_objects_trees = dict()

for tree in set(df['Tree']):
    mean_objects_trees[tree] = df[df['Tree']==tree].mean()[1:]

Считаем средние значения трахеид по всем записям

In [ ]:
global_mean = df.mean()[1:]

Строим таблицы объектов для метода A -- кластеризация отклонений средних объектов по году от среднего глобального объекта:

Метод А:

1. $R^A(y)=\frac{1}{\left| T(y) \right|}\sum_{t\in T(y)}R(t,y)$

2. $R_{mean}^A=\frac{1}{\sum_{t\in T}\left| Y(t)\right|}\sum_{t\in T}\sum_{y\in Y(t)}R(t,y)$

3. $O_A(y)=\frac{R^A(y)}{R_{mean}^A}$

Метод А:

1. $R^A(y)=\frac{1}{\left| T \right|}\sum_{t\in T}R(t,y)$

2. $R_{mean}^A=\frac{1}{\sum_{t\in T}\left| Y(t)\right|}\sum_{t\in T}\sum_{y\in Y(t)}R(t,y)$

3. $O_A(y)=\frac{R^A(y)}{R_{mean}^A}$

In [ ]:
quotient_deviation_df_A = []

_columns = {_:f'D{_}' if _<16 else f'CWT{_-15}' for _ in  range(1,31)}
_columns[0] = 'Year'

for year, mean_obj in mean_objects_years.items():
    quotient_deviation_df_A += [[year] + list(mean_obj/global_mean)]

quotient_deviation_df_A = pd.DataFrame(quotient_deviation_df_A).rename(columns=_columns)

In [ ]:
quotient_deviation_df_A.to_excel('../output/quotient_deviation_df_A.xlsx', index=False)

Строим таблицы объектов для метода B -- кластеризация средних отклонений объектов по году от среднего объекта по дереву:

Метод B:

1. $R^B(t)=\frac{1}{\left| Y(t) \right|}\sum_{y\in Y(t)}R(t,y)$

2. $o_B(t,y)=\frac{R(t,y)}{R^B(t)}$

3. $O_B(y)=\frac{1}{\left| T(y) \right|}\sum_{t\in T(y)}o_B(t,y)$

In [ ]:
qd_df_B = []

quotient_deviation_df_B = dict()


for _, row in df.iterrows():
    qd_df_B += [[row[0], row[1]] + list(row[2:] / mean_objects_trees[row[0]])]

qd_df_B = pd.DataFrame(qd_df_B).rename(columns=columns)

for year in set(df['Year']):
    temp_data_q = qd_df_B[qd_df_B['Year']==year]
    if len(temp_data_q) > 3:
        quotient_deviation_df_B[year] = temp_data_q.mean()[1:]

quotient_deviation_df_B = pd.DataFrame(quotient_deviation_df_B).transpose()

In [ ]:
quotient_deviation_df_B.to_excel('../output/quotient_deviation_df_B.xlsx', index=True)